# run experiments
for each dataset, for each group size and for each group
get items from the mf for each member of the group
these items will be the input of the algorithm

In [1]:
import numpy as np
import pandas as pd

In [2]:
from typing import List


u_features = np.load('../datasets/movie_lens/mf/U_features.npy')
i_features = np.load('../datasets/movie_lens/mf/I_features.npy')
print(u_features.shape)
print(i_features.shape)

def get_items_for_user(user_id):
    items_ratings = u_features[:, user_id] @ i_features
    items_ids_w_ratings = [(item_id, rating) for item_id, rating in enumerate(items_ratings)]
    items_ids_w_ratings.sort(key=lambda x: x[1], reverse=True)
    return items_ids_w_ratings

def get_items_for_users(users_id: List):
    items_ratings = i_features.T @ u_features[:, users_id]
    return items_ratings
    
ratings = get_items_for_users([10,20,30])
ratings.shape


(200, 162541)
(200, 59047)


(59047, 3)

In [3]:
import itertools

for a, b in itertools.combinations(range(3) , 2):
    print(a, b)

0 1
0 2
1 2


In [26]:
from collections import defaultdict
import itertools
import math

from tqdm import tqdm


def select_top_n(score_list, top_n):
    top_n_ind = np.argpartition(score_list, -top_n)[-top_n:]
    sorted_top_n_indicies = top_n_ind[np.argsort(-score_list[top_n_ind])]
    return sorted_top_n_indicies

def get_idx_of_top_n(score_list, top_n):
    """Returns the indices of the top_n items"""
    top_n_idx = np.argpartition(score_list, -top_n)[-top_n:]
    return top_n_idx

def avg_algorithm(group_items, top_n: int):
    """
    Returns items ordered by average rating.
    """
    means = group_items.mean(axis=1)
    top_n_idx = select_top_n(means, top_n)
    return top_n_idx

def lm_algorithm(group_items, top_n: int):
    """
    Returns items ordered by least min value across user rating.
    """
    mins = group_items.min(axis=1)
    top_n_idx = select_top_n(mins, top_n)
    return top_n_idx

def fai_algorithm(group_items, top_n: int):
    """
    Returns items ordered by max of users each one by one per turn.
    So first item is selected as max of first user, second item by second and so on...
    """
    group_size = group_items.shape[0]
    # apply select_top_n to each user
    top_n_required_per_user = math.ceil(top_n / group_size)
    top_n_idx_per_user = np.apply_along_axis(lambda row: select_top_n(row, top_n_required_per_user), 1, group_items)
    # flatten the list to get the turn by turn top_n_idx
    top_n = top_n_idx_per_user.flatten(order='F')[:top_n]
    return top_n

def get_rank(score_list_np):
    """Best item has a rank of 1"""
    ranks = np.zeros(score_list_np.shape, dtype=np.int32)
    ranks[score_list_np.argsort()] = np.arange(start=score_list_np.shape[0], stop=0, step=-1)
    return ranks

def xpo_algorithm(group_items, top_n: int, n_candidates:int, type:str='XPO', mc_trials:int=1000):
    if type != 'XPO' and type != 'NPO':
        raise ValueError(f'Unknown type: {type}, only XPO and NPO are supported')

    group_size = group_items.shape[1]
    # first select top candidates for each member
    top_candidates_ids_per_member = np.apply_along_axis(lambda u_items: select_top_n(u_items, n_candidates), 0, group_items)
    # these are the original items ids
    top_candidates_idx = np.array(sorted(set(top_candidates_ids_per_member.flatten())))

    # get the candidate group items for each member
    candidate_group_items = group_items[top_candidates_idx, :] # this is the first id mapping (to go back to original, index by top_candidates_idx)

    # from now on, item ids are ids into this candidate group items list

    # calculate the rank of candidates and all values bigger than n_candidates are set to n_candidates + 1
    # ranks will therefore be [1, 101]
    rank_of_candidates = np.apply_along_axis(get_rank, 0, candidate_group_items)
    rank_of_candidates = np.minimum(rank_of_candidates, n_candidates + 1)

    # now we want to compare all pairs of items to calculate the their Pareto optimality level
    # we say that item a is pareto optimal over b if for all group members rank(a) <= rank(b)
    # they can be both pareto optimal(if their ranks are equal)
    # or non-pareto optimal(if their dominate each other for different group members)
    number_of_items = len(candidate_group_items)
    # print(f'Number of items: {number_of_items}')
    pareto_levels = [1] * number_of_items # same ids as candidate_group_items
    for a, b in itertools.combinations(range(len(candidate_group_items)) , 2):
        rank_of_candidates_a = rank_of_candidates[a]
        rank_of_candidates_b = rank_of_candidates[b]

        rank_dif = rank_of_candidates_a - rank_of_candidates_b
        # for each group member, if a is pareto optimal over b, then all rank_difs will be negative or zero
        # if b is pareto optimal over 1, then all rank_difs will be positive or zero
        # and atleast one rank for any group member will negative or positive (non-zero) respectively
        a_paretooptim = np.all(rank_dif <= 0) & np.any(rank_dif < 0)
        b_paretooptim = np.all(rank_dif >= 0) & np.any(rank_dif > 0)
        # the lower the level the better, as an item, if you are bester (dominated) we lower your level by 1
        # where level 1 is best and level infinity is worst
        if a_paretooptim:
            pareto_levels[b] += 1
        if b_paretooptim:
            pareto_levels[a] += 1

    # now group the items by pareto level
    pareto_levels_pd = pd.DataFrame(enumerate(pareto_levels), columns=['item_id', 'pareto_level'])
    pareto_levels_pd = pareto_levels_pd.sort_values(by='pareto_level', ascending=True)

    # select candidates by pareto level, cut off at pareto_level = top_n
    if type == 'NPO':
        final_candidates = pareto_levels_pd[pareto_levels_pd['pareto_level'] <= top_n]['item_id'].explode().to_numpy()

    # select candidates starting at pareto level 1 and cut off when the number of candidates is equal or greater than top_n
    if type == 'XPO':
        # add cumulative count of items to the grouped dataframe
        pareto_levels_grouped = pareto_levels_pd.groupby('pareto_level').agg(level=('pareto_level', 'first'), items=('item_id','unique'), items_count=('item_id','count'))
        pareto_levels_grouped['cum_items_count'] = pareto_levels_grouped['items_count'].cumsum()
        idx_of_first_larger_than_top_k = (pareto_levels_grouped['cum_items_count'] > top_n).idxmax()
        # select all pareto levels that will get us top_n items and explode the lists to get the top_n items
        final_candidates = pareto_levels_grouped.iloc[0:idx_of_first_larger_than_top_k]['items'].explode().to_numpy()

    final_candidates_group_items = group_items[final_candidates, :] # second id mapping (to go back index by final_candidates)
    
    # simple Monte Carlo method for computing probabilities of linear aggregation strategy ratios (analytical solution not feasible)
    # now we want to try many different weights of users and award points to winners based on the weighted ranks

    mc_score = np.zeros(len(final_candidates))

    for i in range(mc_trials):
        # get random weights summing up to 1 for the group members
        weights = np.random.random(group_size)
        weights /= weights.sum()
        
        group_items_score = np.sum(weights * final_candidates_group_items, axis=1)
        top_n_idx = get_idx_of_top_n(group_items_score, top_n)
        mc_score[top_n_idx] += 1

    # print(f'MC score: {mc_score}')
    top_n_idx = get_idx_of_top_n(mc_score, top_n)
    # print(top_n_idx)
    # print(final_candidates[top_n_idx])
    # now we need to get the original item ids from the final_candidates list and then top_candidates_idx
    final_top_candidates = top_candidates_idx[final_candidates[top_n_idx]]
    # print(f'Final top candidates: {final_top_candidates}')
    return final_top_candidates


group_size = 5

# load groups
groups = pd.read_csv('../notebooks/dfs/groups/kgrec/top_k_10.csv')
#concatenate first 5 columns to array of ints
groups = groups.iloc[:,:group_size].values
# print(groups.shape)
# print(groups[0:2])

# group_items = []
# for group in groups:
#     members = {}
#     member_items
#     for member in group:
#         items = get_items_for_users(member)
#         members[member] = items
#     group_items.append(members)s
#     break


# first_group = group_items[0]
# for member, items in first_group.items():
#     print(member, items[0], items[-1])
#     break

rec_it_avg = []
rec_it_lm = []
rec_it_fai = []
rec_it_xpo = []

for group_members in tqdm(groups):
    items = get_items_for_users(group_members)

    # # avg_algorithm
    # top_n_items_avg = avg_algorithm(items, 10)
    # rec_it_avg.append(top_n_items_avg)

    # # lm_algorithm
    # top_n_items_lm = lm_algorithm(items, 10)
    # rec_it_lm.append(top_n_items_lm)

    # # fai_algorithm
    # top_n_items_fai = fai_algorithm(items, 10)
    # rec_it_fai.append(top_n_items_fai)

    # xpo_algorithm
    top_n_items_xpo = xpo_algorithm(items, 10, 100, type='NPO')
    rec_it_xpo.append(top_n_items_xpo)
    



  5%|▌         | 52/1000 [00:53<16:13,  1.03s/it]


KeyboardInterrupt: 

In [ ]:
x = [10, 9, 5, 6] # expected [2,3,1,0]
np.array(x).argsort()

In [ ]:
x = [18, 1, 2, 5, 6], [1, 2, 3, 4, 5] # expected [12340]

def get_rank(score_list_np):
    ranks = np.zeros(score_list_np.shape, dtype=np.int32)
    ranks[score_list_np.argsort()] = np.arange(start=score_list_np.shape[0], stop=0, step=-1)
    print(ranks)

np.apply_along_axis(get_rank, 1, np.array(x))

In [ ]:
np.expand_dims(np.arange(10), axis=1).repeat(3, axis=1)